In [1]:
import tensorflow as tf#载入tf
import numpy as np#numpy我不知道用来干什么的
class MNISTLoader():#对数据载入的类
    def __init__(self):#默认
        mnist = tf.keras.datasets.mnist#貌似tf中就包含了mnist手写体
        (self.train_data,self.train_label),(self.test_data,self.test_label) =mnist.load_data()#载入数据
        #分别载入训练数据和测试数据 其中label是标签。
        self.train_data = np.expand_dims(self.train_data.astype(np.float32)/255.0,axis=-1)
        #对训练数据进行处理，图像好像用0-255表示，除以255的话，把数据控制到（0.255）
        #当然因为Python对浮点数表示是近似的，这一个步骤是有可能产生一定误差的
        
        #np.expand_dims用于扩展数组的形状，扩展了一维数组
        #astype：转换数组的数据类型，转换为float格式 
        #对测试数据进行处理
        self.test_data = np.expand_dims(self.test_data.astype(np.float32)/255.0,axis =-1)
       #下面的代码应该是对train——label和test-lebel进行格式转换。
       #转换成整数类型。
        self.train_label = self.train_label.astype(np.int32)
        self.test_label = self.test_label.astype(np.int32)
        #shape[0]就是读取矩阵第一维度的长度
        self.num_train_data, self.num_test_data = self.train_data.shape[0],self.test_data.shape[0]
        
    def get_batch(self,batch_size):
        index = np.random.randint(0,np.shape(self.train_data)[0], batch_size)
        #随机数，随机取
        return self.train_data[index,:],self.train_label[index]

#前面是对数据的处理，现在是模型的构建。
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        #展平一个张量。Flatten层用来将输入“压平”，即把多维的输入一维化，常用在从卷积层到全连接层的过渡。Flatten不影响batch的大小。
        #多维数组输出成一维的

        self.dense1 = tf.keras.layers.Dense(units=100,activation=tf.nn.relu)#Dense层(全连接层）
        #units：大于0的整数，代表该层的输出维度。
        #输出100维度
        #activation：激活函数，为预定义的激活函数名（参考激活函数），
        self.dense2 = tf.keras.layers.Dense(units=10)
        #这里没有设置输入维度
    
    
    #dense1 将784维的数据输出成100维
    #dense2 将100维的数据输出成10维。（对应10个数字）
    def call(self,inputs):
        x = self.flatten(inputs)#对数据进行展开
        x = self.dense1(x)#对数据进行第一层全连接处理
        x = self.dense2(x)#对数据进行第二层全连接处理
        output = tf.nn.softmax(x)#这里应该是搞成概率。
        #Softmax的含义：Softmax简单的说就是把一个N*1的向量归一化为（0，1）之间的值。
        #由于其中采用指数运算，使得向量中数值较大的量特征更加明显。
        return output
        #返回输出
        
#模型的训练
num_epochs = 5
batch_size = 50
learning_rate = 0.001
model = MLP()
#建立模型model
data_loader = MNISTLoader()
#建立数据。
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
#Adam优化器，速度设置为learning_rate
num_batches = int(data_loader.num_train_data // batch_size * num_epochs)

for batch_index in range(num_batches):
    X,y = data_loader.get_batch(batch_size)#将测试数据传输给X，label传递给y
    with tf.GradientTape() as tape:#梯度求解利器
        y_pred = model(X)#传入模型中，求梯度
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
        #前面提到过，y中的是label，应该是标签数据。
        #这里的y_pred是模型算出来的，y是标签数据。
        loss = tf.reduce_mean(loss)
        #tf.reduce_mean 函数用于计算张量tensor沿着指定的数轴（tensor的某一维度）上的的平均值，主要用作降维或者计算tensor（图像）的平均值。
        #损失函数。把缺失值和事实值进行比较，求出缺失函数。
        print("batch %d; loss %f" %(batch_index,loss.numpy()))
        #对数据结果进行输出。batch 第几个数据。loss：损失数据。
    grads = tape.gradient(loss,model.variables)
    #求导，loss关于model.variables的导数。
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))
    #更新模型。
#这以上都是循环中的。下面是对模型的评估。
    
sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
#metrics，它用来在训练过程中监测一些性能指标，而这个性能指标是什么可以由我们来指定。指定的方法有两种：
num_batches = int(data_loader.num_test_data //batch_size)
for batch_index in range(num_batches):
    #使用for循环迭代分批次传入测试集数据的预测结果和真实结果。
    start_index ,end_index = batch_index * batch_size, (batch_index +1 )*batch_size
    y_pred = model.predict(data_loader.test_data[start_index:end_index])#向模型中传入测试样本。
    sparse_categorical_accuracy.update_state(
        y_true=data_loader.test_label[start_index:end_index],y_pred=y_pred)#向评估器中输入y_pred和y_true
print("test accuracy:%f" %sparse_categorical_accuracy.result())
    #输出分析器结果。

#这个MNIST测试，通过python自行下载数据，之后，通过MNISTLoader类，对数据进行预加载，准备好测试集，测试集的标签，验证集以及验证集的标签
#在设计模型的时候，通过继承tf.keras.Model来创建模型，用flatten（）将数据展平，之后第一层dense将784维数据转成100维。
#之后第二次dense将数据转化成10维的数据，和标签进行对比。计算损失函数。对模型进行修正。

batch 0; loss 2.435704
batch 1; loss 2.281685
batch 2; loss 2.219429
batch 3; loss 2.178443
batch 4; loss 2.254947
batch 5; loss 1.913177
batch 6; loss 1.959704
batch 7; loss 1.950780
batch 8; loss 1.982052
batch 9; loss 1.724670
batch 10; loss 1.634899
batch 11; loss 1.731970
batch 12; loss 1.613028
batch 13; loss 1.632538
batch 14; loss 1.580789
batch 15; loss 1.528704
batch 16; loss 1.533174
batch 17; loss 1.422002
batch 18; loss 1.476742
batch 19; loss 1.350432
batch 20; loss 1.437230
batch 21; loss 1.204467
batch 22; loss 1.247509
batch 23; loss 1.156559
batch 24; loss 1.314664
batch 25; loss 1.177277
batch 26; loss 0.926807
batch 27; loss 1.082734
batch 28; loss 1.101030
batch 29; loss 0.948216
batch 30; loss 0.993323
batch 31; loss 0.887430
batch 32; loss 0.809396
batch 33; loss 1.030158
batch 34; loss 0.832521
batch 35; loss 0.855834
batch 36; loss 0.614965
batch 37; loss 0.793893
batch 38; loss 0.688177
batch 39; loss 0.682397
batch 40; loss 0.655827
batch 41; loss 0.770653
ba

batch 336; loss 0.149060
batch 337; loss 0.487112
batch 338; loss 0.375629
batch 339; loss 0.379030
batch 340; loss 0.239529
batch 341; loss 0.101051
batch 342; loss 0.141217
batch 343; loss 0.330567
batch 344; loss 0.370381
batch 345; loss 0.323339
batch 346; loss 0.179524
batch 347; loss 0.317260
batch 348; loss 0.635635
batch 349; loss 0.254525
batch 350; loss 0.234543
batch 351; loss 0.327555
batch 352; loss 0.154215
batch 353; loss 0.195563
batch 354; loss 0.377777
batch 355; loss 0.410707
batch 356; loss 0.246019
batch 357; loss 0.594048
batch 358; loss 0.310890
batch 359; loss 0.285617
batch 360; loss 0.275198
batch 361; loss 0.254959
batch 362; loss 0.120895
batch 363; loss 0.185042
batch 364; loss 0.383080
batch 365; loss 0.288152
batch 366; loss 0.284752
batch 367; loss 0.418601
batch 368; loss 0.404565
batch 369; loss 0.207572
batch 370; loss 0.508120
batch 371; loss 0.364130
batch 372; loss 0.194773
batch 373; loss 0.278381
batch 374; loss 0.269186
batch 375; loss 0.172217


batch 667; loss 0.185425
batch 668; loss 0.185919
batch 669; loss 0.328525
batch 670; loss 0.207061
batch 671; loss 0.073132
batch 672; loss 0.294314
batch 673; loss 0.159262
batch 674; loss 0.204907
batch 675; loss 0.171947
batch 676; loss 0.299081
batch 677; loss 0.145088
batch 678; loss 0.310765
batch 679; loss 0.253192
batch 680; loss 0.123281
batch 681; loss 0.366771
batch 682; loss 0.149313
batch 683; loss 0.091537
batch 684; loss 0.266915
batch 685; loss 0.146667
batch 686; loss 0.215565
batch 687; loss 0.296245
batch 688; loss 0.195277
batch 689; loss 0.223575
batch 690; loss 0.196736
batch 691; loss 0.217559
batch 692; loss 0.269893
batch 693; loss 0.253466
batch 694; loss 0.303888
batch 695; loss 0.129823
batch 696; loss 0.064558
batch 697; loss 0.121141
batch 698; loss 0.155445
batch 699; loss 0.200917
batch 700; loss 0.200411
batch 701; loss 0.201437
batch 702; loss 0.090635
batch 703; loss 0.215705
batch 704; loss 0.154309
batch 705; loss 0.063740
batch 706; loss 0.122036


batch 1005; loss 0.076931
batch 1006; loss 0.107653
batch 1007; loss 0.173161
batch 1008; loss 0.084290
batch 1009; loss 0.054651
batch 1010; loss 0.118724
batch 1011; loss 0.359910
batch 1012; loss 0.186781
batch 1013; loss 0.208767
batch 1014; loss 0.158194
batch 1015; loss 0.082629
batch 1016; loss 0.175895
batch 1017; loss 0.178980
batch 1018; loss 0.300604
batch 1019; loss 0.200617
batch 1020; loss 0.277675
batch 1021; loss 0.277029
batch 1022; loss 0.113531
batch 1023; loss 0.324472
batch 1024; loss 0.047499
batch 1025; loss 0.092025
batch 1026; loss 0.288830
batch 1027; loss 0.141982
batch 1028; loss 0.176824
batch 1029; loss 0.086257
batch 1030; loss 0.270853
batch 1031; loss 0.394003
batch 1032; loss 0.119416
batch 1033; loss 0.082288
batch 1034; loss 0.128065
batch 1035; loss 0.263732
batch 1036; loss 0.097338
batch 1037; loss 0.423558
batch 1038; loss 0.092135
batch 1039; loss 0.118686
batch 1040; loss 0.100412
batch 1041; loss 0.048520
batch 1042; loss 0.136102
batch 1043; 

batch 1347; loss 0.267812
batch 1348; loss 0.324396
batch 1349; loss 0.146309
batch 1350; loss 0.162421
batch 1351; loss 0.133394
batch 1352; loss 0.077476
batch 1353; loss 0.139959
batch 1354; loss 0.091440
batch 1355; loss 0.091782
batch 1356; loss 0.151934
batch 1357; loss 0.146269
batch 1358; loss 0.114968
batch 1359; loss 0.165681
batch 1360; loss 0.144344
batch 1361; loss 0.135857
batch 1362; loss 0.112335
batch 1363; loss 0.110075
batch 1364; loss 0.123974
batch 1365; loss 0.256087
batch 1366; loss 0.145964
batch 1367; loss 0.116216
batch 1368; loss 0.061856
batch 1369; loss 0.207959
batch 1370; loss 0.104516
batch 1371; loss 0.062781
batch 1372; loss 0.114169
batch 1373; loss 0.037603
batch 1374; loss 0.185548
batch 1375; loss 0.181562
batch 1376; loss 0.183753
batch 1377; loss 0.084816
batch 1378; loss 0.117144
batch 1379; loss 0.102032
batch 1380; loss 0.087974
batch 1381; loss 0.222726
batch 1382; loss 0.261237
batch 1383; loss 0.115530
batch 1384; loss 0.134968
batch 1385; 

...

batch 4697; loss 0.012422
batch 4698; loss 0.123020
batch 4699; loss 0.049124
batch 4700; loss 0.036446
batch 4701; loss 0.026418
batch 4702; loss 0.017018
batch 4703; loss 0.009559
batch 4704; loss 0.010667
batch 4705; loss 0.033167
batch 4706; loss 0.078176
batch 4707; loss 0.089886
batch 4708; loss 0.062253
batch 4709; loss 0.008690
batch 4710; loss 0.020217
batch 4711; loss 0.045171
batch 4712; loss 0.056857
batch 4713; loss 0.047382
batch 4714; loss 0.029909
batch 4715; loss 0.028293
batch 4716; loss 0.010736
batch 4717; loss 0.050743
batch 4718; loss 0.029378
batch 4719; loss 0.033302
batch 4720; loss 0.064650
batch 4721; loss 0.030525
batch 4722; loss 0.204260
batch 4723; loss 0.119933
batch 4724; loss 0.023915
batch 4725; loss 0.029550
batch 4726; loss 0.224170
batch 4727; loss 0.098509
batch 4728; loss 0.010678
batch 4729; loss 0.067481
batch 4730; loss 0.092538
batch 4731; loss 0.082230
batch 4732; loss 0.023891
batch 4733; loss 0.048378
batch 4734; loss 0.081941
batch 4735; 

batch 5018; loss 0.044267
batch 5019; loss 0.018644
batch 5020; loss 0.014206
batch 5021; loss 0.034699
batch 5022; loss 0.049091
batch 5023; loss 0.016291
batch 5024; loss 0.040105
batch 5025; loss 0.071318
batch 5026; loss 0.094314
batch 5027; loss 0.114621
batch 5028; loss 0.029598
batch 5029; loss 0.011747
batch 5030; loss 0.010999
batch 5031; loss 0.032017
batch 5032; loss 0.038388
batch 5033; loss 0.050225
batch 5034; loss 0.072550
batch 5035; loss 0.018681
batch 5036; loss 0.031347
batch 5037; loss 0.048765
batch 5038; loss 0.235104
batch 5039; loss 0.018464
batch 5040; loss 0.014771
batch 5041; loss 0.107527
batch 5042; loss 0.062806
batch 5043; loss 0.063385
batch 5044; loss 0.126417
batch 5045; loss 0.015709
batch 5046; loss 0.005023
batch 5047; loss 0.020061
batch 5048; loss 0.023953
batch 5049; loss 0.208962
batch 5050; loss 0.013388
batch 5051; loss 0.045478
batch 5052; loss 0.027480
batch 5053; loss 0.233883
batch 5054; loss 0.179173
batch 5055; loss 0.023298
batch 5056; 

batch 5348; loss 0.060194
batch 5349; loss 0.054508
batch 5350; loss 0.012155
batch 5351; loss 0.019818
batch 5352; loss 0.041044
batch 5353; loss 0.014042
batch 5354; loss 0.041744
batch 5355; loss 0.016688
batch 5356; loss 0.077416
batch 5357; loss 0.089829
batch 5358; loss 0.053727
batch 5359; loss 0.011690
batch 5360; loss 0.064893
batch 5361; loss 0.037601
batch 5362; loss 0.011160
batch 5363; loss 0.059876
batch 5364; loss 0.007056
batch 5365; loss 0.138694
batch 5366; loss 0.019626
batch 5367; loss 0.037330
batch 5368; loss 0.019031
batch 5369; loss 0.061553
batch 5370; loss 0.026208
batch 5371; loss 0.082696
batch 5372; loss 0.022636
batch 5373; loss 0.052603
batch 5374; loss 0.015165
batch 5375; loss 0.010915
batch 5376; loss 0.025633
batch 5377; loss 0.031632
batch 5378; loss 0.052314
batch 5379; loss 0.230201
batch 5380; loss 0.038181
batch 5381; loss 0.117386
batch 5382; loss 0.060224
batch 5383; loss 0.079975
batch 5384; loss 0.034995
batch 5385; loss 0.190434
batch 5386; 

batch 5679; loss 0.136244
batch 5680; loss 0.021992
batch 5681; loss 0.110605
batch 5682; loss 0.009042
batch 5683; loss 0.043087
batch 5684; loss 0.063698
batch 5685; loss 0.023325
batch 5686; loss 0.006495
batch 5687; loss 0.039989
batch 5688; loss 0.018498
batch 5689; loss 0.132519
batch 5690; loss 0.041835
batch 5691; loss 0.010091
batch 5692; loss 0.045624
batch 5693; loss 0.204283
batch 5694; loss 0.023769
batch 5695; loss 0.045677
batch 5696; loss 0.056393
batch 5697; loss 0.016706
batch 5698; loss 0.078045
batch 5699; loss 0.036542
batch 5700; loss 0.113931
batch 5701; loss 0.037345
batch 5702; loss 0.040014
batch 5703; loss 0.079411
batch 5704; loss 0.027556
batch 5705; loss 0.027432
batch 5706; loss 0.193435
batch 5707; loss 0.074650
batch 5708; loss 0.111021
batch 5709; loss 0.005289
batch 5710; loss 0.024497
batch 5711; loss 0.052575
batch 5712; loss 0.033299
batch 5713; loss 0.019172
batch 5714; loss 0.250133
batch 5715; loss 0.038243
batch 5716; loss 0.032094
batch 5717; 

test accuracy:0.973700
